### Integrating into FeniCSx

A common use case for the random fields generated by `parafields` is to use them as parameters in PDE models. Examples of such models could e.g. be in subsurface flow, where the exact spatial details of the soil permeability are known, but some geostatistic properties are known from measurements and soil characteristics studies. This notebook shows how such simulation is run using the popular [FeniCSx framework](https://fenicsproject.org) for the numerical solution of PDEs with the Finite Element Method. The shown code is an adaption of [their tutorial on the Poisson equation](https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_poisson.html) - check it out if you need more information about FeniCSx. Note: As `fenicsx` requires MPI, you need to use the parallel version of `parafields` for this notebook.

Throughout this notebook, we will solve the Poisson equation with a heterogeneous coefficient $k(\mathbf{x})$:

$$
\begin{align}
-\nabla\cdot\left(k(\mathbf{x})\nabla u(\mathbf{x})\right) & = 0 \ \ && \forall\mathbf{x}\in\Omega \\
u(\mathbf{x}) & = 1 - x && \forall\textbf{x}\in\partial\Omega
\end{align}
$$

on the unit square $\Omega=(0,1)^2$.

In [ ]:
from mpi4py import MPI
import parafields
import numpy as np

from dolfinx import fem, mesh, plot
from ufl import dx, grad, inner, TrialFunction, TestFunction

from petsc4py.PETSc import ScalarType

import pyvista

We start off by defining a log-normal distributed random field:

In [ ]:
field = parafields.generate_field(
    cells=(512, 512), extensions=(1.0, 1.0), transform="lognormal", dtype=ScalarType
)

Next, we create some basic building blocks a FeniCSx program: A triangular mesh, a function space using continuous linear Lagrange elements and some boundary conditions for the left and right hand face of the unit cube domain.

In [ ]:
msh = mesh.create_rectangle(
    comm=field.comm,
    points=((0.0, 0.0), field.extensions),
    n=field.cells,
    cell_type=mesh.CellType.triangle,
)

In [ ]:
V = fem.FunctionSpace(msh, ("Lagrange", 1))

In [ ]:
lfacets = mesh.locate_entities_boundary(
    msh, dim=1, marker=lambda x: np.isclose(x[0], 0.0)
)
ldofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=lfacets)
lbc = fem.dirichletbc(value=ScalarType(1), dofs=ldofs, V=V)

In [ ]:
rfacets = mesh.locate_entities_boundary(
    msh, dim=1, marker=lambda x: np.isclose(x[0], 1.0)
)
rdofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=rfacets)
rbc = fem.dirichletbc(value=ScalarType(0), dofs=rdofs, V=V)

Now, to integrate our random field into the PDE, we create a FeniCS function object from our random field by passing it the function space that it should be interpolated into:

In [ ]:
k = field.fenicsx_function(V)

Defining test and trial functions, as well as the bilinear form:

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
a = inner(k * grad(u), grad(v)) * dx
L = 1e-16 * v * dx  # TODO: How does FeniCSx allow the RHS to be zero?

We now solve the arising linear problem...

In [ ]:
problem = fem.petsc.LinearProblem(
    a, L=L, bcs=[lbc, rbc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"}
)
uh = problem.solve()

... and visualize the results with `pyvista`:

In [ ]:
import pyvista

cells, types, x = plot.create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(cells, types, x)
grid.point_data["u"] = uh.x.array.real
grid.set_active_scalars("u")
plotter = pyvista.Plotter()
warped = grid.warp_by_scalar()
plotter.add_mesh(warped)
plotter.show()